# CSCC11 - Introduction to Machine Learning, Fall 2022, Assignment 1

In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import statistics
from scipy import stats

In [2]:
#TO-DO
"""
Read the csv file into a DataFrame - df
"""
df = pd.read_csv('Admission_Predict.csv', sep=',', header=0)

In [3]:
"""
Print the DataFrame
"""
df

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65
...,...,...,...,...,...,...,...,...,...
395,396,324,110,3,3.5,3.5,9.04,1,0.82
396,397,325,107,3,3.0,3.5,9.11,1,0.84
397,398,330,116,4,5.0,4.5,9.45,1,0.91
398,399,312,103,3,3.5,4.0,8.78,0,0.67


In [4]:
#TO-DO
"""
Print the length of the DataFrame.
Print the column names of the DataFrame.
"""
print(df.shape[0])
print(df.columns.values)

400
['Serial No.' 'GRE Score' 'TOEFL Score' 'University Rating' 'SOP' 'LOR '
 'CGPA' 'Research' 'Chance of Admit ']


In [5]:
#TO-DO
"""
Define an “X” array that would hold our independent features for regression purposes.  
Define a "Y" array that would hold our target variable.

Print the shape of both the arrays.
"""
X = df.iloc[:,:-1]
Y = df.iloc[:,-1].values.reshape(X.shape[0],1)
print(X.shape)
print(Y.shape)

(400, 8)
(400, 1)


## Split the data 

In [6]:
#TO-DO
"""
Split the dataset into train dataset and test dataset.
Set the random state to any number in order to maintain consistency while generating random numbers over several runs.
"""
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=10)

# Linear Regression

In [7]:
#TO-DO
def find_optimal_parameters(x, y):
    """ Compute closed form solution for linear regression!
    Optimal weight w* in linear regression is given by w* = (X^T X)^(-1) X^T Y
    
    Args:
    - x (ndarray (Shape: (N, D))): A NxD matrix corresponding to the inputs.
    - y (ndarray (Shape: (N, 1))): A N-column vector corresponding to the outputs given the inputs.
    
    Output:
    - w (ndarray (Shape: (D, 1))): A D-column vector corresponding to the bias and weights of the linear model.
    """
    # Pad 1's for the bias term, Why?
    x = np.hstack((np.ones((x.shape[0], 1)), x))

    # Note that we could use pseudoinverse here instead: np.linalg.pinv
    # @ is alias for matmul
    p1 = np.linalg.pinv(np.transpose(x) @ x)# (X^T X) inverse
    p2 = np.transpose(x) @ y # X^T Y
    w = p1 @ p2
    return w

### Train linear regression model using training data

In [8]:
#TO-DO
def get_pred_Y(trained_w, X_pred):
    """ Return predicted Y
    Args:
    - trained_w (ndarray (Shape: (D+1, 1))): A (D+1)x1 column vector containing linear regression weights.
    - X_pred (ndarray (Shape: (N, D))): A NxD matrix corresponding to the prediction inputs.
    
    Output:
    - pred_Y (ndarray (Shape: (N, 1))): A Nx1 column vector corresponding to the predicted outputs.
    """
    pad_Y     = np.hstack((np.ones((X_pred.shape[0], 1)), X_pred))
    pred_Y    = pad_Y @ trained_w
    return pred_Y

In [9]:
#TO-DO
def get_mae(Y_truth, Y_pred):
    """ Return Mean absolute error
    Args:
    - Y_truth (ndarray (Shape: (N, 1))): A Nx1 column vector corresponding to the actual outputs.
    - Y_pred (ndarray (Shape: (N, 1))): A Nx1 column vector corresponding to the predicted outputs.
    
    Output:
    - MSE (ndarray (Shape: (1,))).
    """
    'check if both inputs are of the same shape'
    if (Y_truth.shape != Y_pred.shape):
        print("Warming!",Y_truth.shape, Y_pred.shape)
        return -1
    mae    = np.sum(np.absolute(Y_truth - Y_pred))/Y_pred.shape[0]
    return mae

def get_mse(Y_truth, Y_pred):
    """ Return Mean squared error
    Args:
    - Y_truth (ndarray (Shape: (N, 1))): A Nx1 column vector corresponding to the actual outputs.
    - Y_pred (ndarray (Shape: (N, 1))): A Nx1 column vector corresponding to the predicted outputs.
    
    Output:
    - MSE (ndarray (Shape: (1,))).
    """
    'check if both inputs are of the same shape'
    if (Y_truth.shape != Y_pred.shape):
        print("Warming!",Y_truth.shape, Y_pred.shape)
        return -1
    mse    = np.sum(np.square(Y_truth - Y_pred))/Y_pred.shape[0]
    return mse

### Get predictions on train data

In [10]:
w_optimal = find_optimal_parameters(X_train, Y_train)
print(w_optimal)

[[-1.37559131e+00]
 [ 1.31968750e-04]
 [ 2.29768983e-03]
 [ 2.57080482e-03]
 [ 1.10083142e-02]
 [-7.48574188e-04]
 [ 2.06096386e-02]
 [ 1.11998704e-01]
 [ 1.24114883e-02]]


In [11]:
pred_Y    = get_pred_Y(w_optimal, X_train)
print('train error (MSE): ', get_mse(Y_train, pred_Y))
print('train error (MAE): ', get_mae(Y_train, pred_Y))

train error (MSE):  0.0033854592598648427
train error (MAE):  0.042913770093862096


### Get predictions and performance on test data

In [12]:
pred_Y    = get_pred_Y(w_optimal, X_test)
print('test error (MSE):: ', get_mse(Y_test, pred_Y))
print('test error (MAE): ', get_mae(Y_test, pred_Y))

test error (MSE)::  0.00452619068784237
test error (MAE):  0.04690716026358087


# Silouette Coefficient

In [13]:
#TO-DO
n_silhouette = []

kmeans_kwargs= {
    "init":"k-means++",
    "n_init":30,
    "max_iter":250,
    "random_state":2
}

"""
Perform the following steps:

1. Loop over the various possible K values you wish to test
2. Initialize a K means object.
3. Fit the training data on the K means object.
4. Use the silhouette score method available from the sklearn metrics.
5. Append the score to the silhouetter_coefficients list.
6. Display the the silhouette coefficient associated with each value of K.
"""
for K in range(2,11):
    kmeans = KMeans(n_clusters=K, init=kmeans_kwargs["init"], n_init=kmeans_kwargs["n_init"], 
    max_iter=kmeans_kwargs["max_iter"], random_state = kmeans_kwargs["random_state"])
    kmeans.fit_predict(X_train)
    score = silhouette_score(X_train, kmeans.labels_)
    n_silhouette.append(score)
    print("K = ",K, ", silhouetter score = ",score)


    

K =  2 , silhouetter score =  0.6116585331973866
K =  3 , silhouetter score =  0.5439603909593356
K =  4 , silhouetter score =  0.5244267025733921
K =  5 , silhouetter score =  0.48291167894004466
K =  6 , silhouetter score =  0.46041825313273566
K =  7 , silhouetter score =  0.4429618727806108
K =  8 , silhouetter score =  0.41809916954838816
K =  9 , silhouetter score =  0.3988447362932631
K =  10 , silhouetter score =  0.3826793259548973


# K Means

In [14]:
kmeans = KMeans(
    init="k-means++",
    n_clusters = 2, #Input the value you configured using the Silhouette coefficient analysis.
    n_init=30,
    max_iter=250,
    random_state=2
)

#TO-DO
# Fit to the training data
kmeans.fit_predict(X_train)

#TO-DO
# Add the features and the training data you used to the variable below.
training_df_clustered = X_train.copy()

#TO-DO
# Predict clusters for the training data
train_cluster = kmeans.predict(X_train)

#TO-DO
# Add the target and predicted clusters to the training DataFrame
training_df_clustered.insert(loc=8,column="Chance of Admit",value=Y_train)
training_df_clustered.insert(loc=9,column="cluster",value=train_cluster)


#TO-DO
# Set the number of clusters based on the silhouette coefficient analysis
number_cluster = kmeans.cluster_centers_.shape[0]

X_train_clusters_df = []
for i in range(number_cluster):
    X_train_clusters_df.append(training_df_clustered[training_df_clustered['cluster']==i])

In [15]:
print(X_train_clusters_df)

[     Serial No.  GRE Score  TOEFL Score  University Rating  SOP  LOR   CGPA  \
399         400        333          117                  4  5.0   4.0  9.66   
279         280        304          102                  2  3.0   4.0  8.73   
260         261        327          108                  5  5.0   3.5  9.13   
380         381        322          104                  3  3.5   4.0  8.84   
360         361        322          110                  3  4.0   5.0  8.64   
..          ...        ...          ...                ...  ...   ...   ...   
395         396        324          110                  3  3.5   3.5  9.04   
221         222        316          110                  3  3.5   4.0  8.56   
369         370        301           98                  1  2.0   3.0  8.03   
320         321        317          106                  3  4.0   3.5  8.50   
265         266        313          102                  3  2.5   2.5  8.68   

     Research  Chance of Admit  cluster  
399     

# Building Linear Regression for our clusters

In [16]:
from sklearn.linear_model import LinearRegression

"""
The number of clusters would be defined by the outcome of the silhouetter coefficient 
Set up the model of Linear Regression by exploring the different parameters: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html
train_clusters_df is a dataframe that contains both the true cluster values and the predicted cluster values. Feel free to change the variable name to something else if you have been following a different naming convention.
"""

#TO-DOfg
# Set the number of clusters based on the silhouette coefficient analysis
number_cluster = 2
obj_cluster = []

for i in range(number_cluster):
    #TO-DO
    # Initialize a Linear Regression object.
    lr = LinearRegression()
    #Get the specific X_train values according to their predicted clusters.
    X_clustered_data = X_train_clusters_df[i][X_train_clusters_df[i].columns[0:8]]
    #Get the specific Y_train values according to their predicted clusters.
    Y_clustered_data = X_train_clusters_df[i]["Chance of Admit"]
    obj_cluster.append(lr.fit(X_clustered_data, Y_clustered_data)) #Replace the underlines with the variable name you used to create the Linear Regression object.

In [17]:
def predict_value(x_test, kmeans, cluster_linear):
  """
  Input: 
  x_test is the test value that you wish to predict on.
  kmeans is the kmeans object that you have finalized to predict on the test dataset.
  cluster_linear is the list of fitted models on different clusters.

  Return:
  linear_pred - linear_pred will be type list with prediction values
  clusters - clusters_pred will be the prediction of clusters using k means.

  Follow these steps:
  1. Predict clusters using K means object on the test data.
  2. Predict regression values using Linear Regression list.
  3. return both the predictions.

  """
  linear_pred = []
  clusters = kmeans.predict(x_test)
  for i in range(0,x_test.shape[0]):
    #print(x_test.iloc[[i]])
    linear_pred.append(cluster_linear[clusters[i]].predict(x_test.iloc[[i]]))
  return linear_pred, clusters

# Final Steps

In [18]:
#Apply the clustering-based linear regression to the test set.
Y_svr_k_means_pred, clusters_test = predict_value(X_test,kmeans,obj_cluster)

In [19]:
print(Y_svr_k_means_pred)

[array([0.67722484]), array([0.77961651]), array([0.63249548]), array([0.55116358]), array([0.54110348]), array([0.66000388]), array([0.9509116]), array([0.92210144]), array([0.91580334]), array([0.66368536]), array([0.8183683]), array([0.87855199]), array([0.73664435]), array([0.59340214]), array([0.92128584]), array([0.91632721]), array([0.48077289]), array([0.76585254]), array([0.8147176]), array([0.77798748]), array([0.91712802]), array([0.76895964]), array([0.78377861]), array([0.74630533]), array([0.72222038]), array([0.70917961]), array([0.63542041]), array([0.94238505]), array([0.63223766]), array([0.73385595]), array([0.768258]), array([0.67107045]), array([0.76423616]), array([0.73237365]), array([0.64997462]), array([0.88591125]), array([0.51295312]), array([0.64836443]), array([0.79314342]), array([0.81765682]), array([0.93739937]), array([0.83999905]), array([0.76552104]), array([0.47666591]), array([0.68179324]), array([0.71060282]), array([0.69865061]), array([0.70738579

In [23]:
print('test error (MSE):: ', get_mse(Y_test, np.array(Y_svr_k_means_pred).reshape(120,-1)))
print('test error (MAE): ', get_mae(Y_test, np.array(Y_svr_k_means_pred).reshape(120,-1)))

test error (MSE)::  0.004189774794233369
test error (MAE):  0.04486064110635149
